In [2]:
import datetime
import pandas as pd
import requests
import io
import json

url = 'http://gis.sutran.gob.pe/alerta_sutran/script_cgm/carga_xlsx.php'
headers = {'Accept': '/',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'en,es;q=0.9,en-GB;q=0.8',
'Connection': 'keep-alive',
'Content-Length': '9',
'Content-type': 'application/x-www-form-urlencoded; charset=UTF-8-SIG',
'Host': 'gis.sutran.gob.pe',
'Origin': 'http://gis.sutran.gob.pe',
'Referer': 'http://gis.sutran.gob.pe/alerta_sutran/',
'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36'}
body = {'tipo':'MAPA'}
req = requests.post(url, headers=headers, data=body)
now = datetime.datetime.now()

my_json = json.load(io.BytesIO(req.content))

restringido = {}
restringido['data'] = my_json['restringido']

interrumpido = {}
interrumpido['data'] = my_json['interrumpido']

with open('restringido.json', 'w') as f:
    json.dump(restringido, f)

with open('interrumpido.json', 'w') as f:
    json.dump(interrumpido, f)

df_r = pd.read_json('restringido.json', encoding = 'utf-8-sig', orient='split')
df_restringido = pd.json_normalize(df_r['properties'])

df_i = pd.read_json('interrumpido.json', encoding = 'utf-8-sig', orient='split')
df_interrumpido = pd.json_normalize(df_i['properties'])

frames = [df_restringido, df_interrumpido]
df_total = pd.concat(frames, ignore_index=True)
df_total = df_total[df_total['motivo'] == 'HUMANO']
df_total = df_total.replace('TRANSITO', 'TRÁNSITO', regex=True)
df_total = df_total.replace('"', '', regex=True)
df_total['style'] = df_total['estado'].str.split(' ').str[1].str.lower()
df_total.to_csv(f'{now}.csv', index=False)
df_total

,item,visualizacion_,tipo_alerta_,estado,fecha_evento,fecha_actualizacion,ubigeo,afectacion,evento,latitud,...,_sort_,_fecha_actualizacion_,cant_vehiculos_detenidos_pasajeros,cant_vehiculos_detenidos_mercancias,fuente,motivo,codigo_via,nombre_carretera,alerta_activa_24_horas,style
2,29,SI,VIGENTE,TRÁNSITO RESTRINGIDO,23/01/2023,06/02/2023 21:00 HORAS,APURIMAC/ABANCAY/PICHIRHUA,KM 417,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-13.884221,...,-13.884221+-73.031517,2023 21:00 HORAS+02+06+,0,0,CONCESIONARIA SURVIAL,HUMANO,PE-30A,CARRETERA NASCA - PUQUIO - ABANCAY,SI,restringido
3,48,SI,VIGENTE,TRÁNSITO RESTRINGIDO,05/01/2023,06/02/2023 21:00 HORAS,APURIMAC/AYMARAES/TORAYA,KM 377,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-14.07238288,...,-14.07238288+-73.25490188,2023 21:00 HORAS+02+06+,0,0,CEOPOL,HUMANO,PE-30A,CARRETERA NASCA - PUQUIO - ABANCAY,SI,restringido
4,274,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,06/01/2023,06/02/2023 21:00 HORAS,MADRE DE DIOS/TAMBOPATA/LAS PIEDRAS,KM 511+800,(PUENTE MAVILA) PARO INDEFINIDO SOLICITANDO NU...,-11.931332,...,-11.931332+-69.11748,2023 21:00 HORAS+02+06+,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
5,165,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,02/02/2023,06/02/2023 21:00 HORAS,CUSCO/LA CONVENCION/ECHARATE,KM 270,(OCONGATE) PARO INDEFINIDO SOLICITANDO NUEVAS ...,-12.744918469501,...,-12.744918469501+-73.37500517445,2023 21:00 HORAS+02+06+,0,0,CEOPOL,HUMANO,PE-28B,CARRETERA QUINUA - QUILLABAMBA - CUSCO,SI,interrumpido
6,273,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,23/01/2023,06/02/2023 21:00 HORAS,MADRE DE DIOS/TAMBOPATA/LABERINTO,KM 382+900,(SECTOR SANTO DOMINGO) PARO INDEFINIDO SOLICIT...,-12.759719,...,-12.759719+-69.561868,2023 21:00 HORAS+02+06+,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,310,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,04/01/2023,06/02/2023 21:00 HORAS,PUNO/CHUCUITO/POMATA,KM 1463,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-16.26885871,...,-16.26885871+-69.29548283,2023 21:00 HORAS+02+06+,0,0,CEOPOL,HUMANO,PE-3S,LONGITUDINAL DE LA SIERRA SUR,SI,interrumpido
67,311,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,04/01/2023,06/02/2023 21:00 HORAS,PUNO/CHUCUITO/ZEPITA,KM 287,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-16.6042637,...,-16.6042637+-69.1857072,2023 21:00 HORAS+02+06+,0,0,CEOPOL,HUMANO,PE-36A,CARRETERA MOQUEGUA - TORATA - MAZOCRUZ - DESAG...,SI,interrumpido
68,318,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,04/01/2023,06/02/2023 21:00 HORAS,PUNO/EL COLLAO/SANTA ROSA,KM 310,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-16.72557,...,-16.72557+-69.67592,2023 21:00 HORAS+02+06+,0,0,CEOPOL,HUMANO,PE-36A,CARRETERA MOQUEGUA - TORATA - MAZOCRUZ - DESAG...,SI,interrumpido
69,317,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,05/01/2023,06/02/2023 21:00 HORAS,PUNO/EL COLLAO/SANTA ROSA,KM 175 AL 213,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-16.7501666,...,-16.7501666+-69.7209405,2023 21:00 HORAS+02+06+,0,0,CEOPOL,HUMANO,PE-36A,CARRETERA MOQUEGUA - TORATA - MAZOCRUZ - DESAG...,SI,interrumpido
